# Build an End-to-End System


# Introduction  
TO BE DRAFTED 

## Setup  
 For more info see [SETUP Instructions](https://github.com/16032022/DeepLearningAI-OpenAI-projects/blob/main/SETUP.md)

In [26]:
import os
import openai
import sys
sys.path.append('../..')


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

**Response chatbot** : _Panel_ is a powerful Python library used for building interactive web applications, dashboards, and visualizations. It allows you to create rich data exploration interfaces without needing to write extensive JavaScript code, making it a great tool for data scientists, analysts, and developers who want to integrate interactive visualizations with Python code.

In [27]:
# !pip install panel
import panel as pn  # GUI
pn.extension()

**Response chatbot** _utils_ TO BE DRAFTED

In [3]:
import utils
#import utils_pr

In order to use the OpenAI library version `1.0.0`, here is the code that you would use instead for the get_completion function: 

In [28]:
client = openai.OpenAI()
#def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    """
    Function to call the OpenAI API with a list of messages.
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,  # Ensure `messages` is passed
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content


## System of chained prompts for processing the user query

In [29]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"

    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = client.moderations.create(input=user_input)
    if debug: print(f"Moderation API response: {response}")

    moderation_output = response.results[0]
    if moderation_output.flagged:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request.", all_messages

    if debug: print("Step 1: Input passed moderation check.")
    
    # Step 2: Extract category and product information
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    
    if debug: print(f"Step 2: Extracted list of products: {category_and_product_list}")

    # Step 3: Look up product information
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print(f"Step 3: Looked up product information: {product_information}")

    # Step 4: Create the `messages` list
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    # Step 4: Pass `messages` to get_completion()
    final_response = get_completion_from_messages(all_messages + messages)
    if debug: print(f"Step 4: Generated response to user question: {final_response}")

    # Add messages to conversation history
    all_messages = all_messages + messages[1:]

    # Step 5: Check the final response with moderation
    response = client.moderations.create(input=final_response)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information.", all_messages

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Model evaluation of the response
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    evaluation_messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    evaluation_response = get_completion_from_messages(messages)
    if debug: print(f"Step 6: Model evaluated the response: {evaluation_response}")

    # Step 7: If the response is sufficient, return it; otherwise escalate to a human
    if "Y" in evaluation_response:
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

# Test the function
user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also tell me about your TVs"
response, _ = process_user_message(user_input, [])
print(response)


Moderation API response: ModerationCreateResponse(id='modr-AB0EEXrrChWX0b8r5vwz63rcNkyoh', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=1.148572118836455e-05, harassment_threatening=1.1941214097532793e-06, hate=1.4963841294957092e-06, hate_threatening=1.9433115028277825e-07, self_harm=1.1009863555955235e-05, self_harm_instructions=9.630271051719319e-06, self_harm_intent=6.685536209261045e-05, sexual=2.845350536517799e-05, sexual_minors=2.6169869670411572e-05, violence=0.00015530144446529448, violence_graphic=2.096803882523090

**Explanation output** TO BE DRAFTED

## Function that collects user and assistant messages over time

In [30]:
import panel as pn  # GUI
pn.extension()

In [31]:
# Initialize Panel extension
pn.extension()

def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''   # Reset input field
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

## Chat with the chatbot!
Note that the system message includes detailed instructions about what the OrderBot should do.

In [33]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'cf3388b9-7169-4d30-8cea-df305cb9a0ef': {'version…

User Input = 


alternatives 

In [35]:
import panel as pn

# Initialize Panel extension
pn.extension()

# Initialize global variables
panels = []  # Collect messages for display
context = [{'role': 'system', 'content': "You are Service Assistant"}]  # Conversation context

# Define input and button widgets
inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

# Function to collect and display messages
def collect_messages(event=None, debug=False):
    global panels, context
    user_input = inp.value  # Get value from TextInput widget
    
    if debug:
        print(f"User Input = {user_input}")
    
    if user_input == "":
        return
    
    inp.value = ''  # Reset input field
    
    # Debug: Print current context before processing
    if debug:
        print(f"Current Context before processing: {context}")
    
    # Process the user's message and update context
    try:
        response, context = process_user_message(user_input, context, debug=debug)
    except Exception as e:
        print(f"Error processing message: {e}")
        response = "Sorry, there was an error processing your message."
    
    if debug:
        print(f"Response: {response}")
        print(f"Updated Context: {context}")
    
    # Append user and assistant messages to the panel layout
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600))
    )
    panels.append(
        pn.Row('Assistant:', pn.pane.HTML(f'<div style="background-color: #F6F6F6;">{response}</div>', width=600))
    )
    
    # Debug: Print current panel state
    if debug:
        print(f"Panels: {panels}")
    
    # Return a Column containing all the conversation panels
    return pn.Column(*panels)

# Event handler for button click
button_conversation.on_click(lambda event: collect_messages(event, debug=True))

# Dashboard layout
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.Column(*panels, height=300),
)

# Display the dashboard
dashboard.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'3a1446c1-7400-4d74-823c-86eabedaa96a': {'version…

User Input = 


In [38]:
import panel as pn

# Initialize Panel extension
pn.extension()

# Initialize global variables
panels = []  # Collect messages for display
context = [{'role': 'system', 'content': "You are Service Assistant"}]  # Conversation context

# Define input and button widgets
inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

# Function to collect and display messages
def collect_messages(event=None, debug=False):
    global panels, context
    user_input = inp.value  # Get value from TextInput widget
    if debug:
        print(f"User Input = {user_input}")
    
    if user_input == "":
        return
    
    inp.value = ''  # Reset input field
    
    # Process the user's message and update context
    response, context = process_user_message(user_input, context, debug=debug)
    context.append({'role': 'assistant', 'content': f"{response}"})
    
    # Append user and assistant messages to the panel layout
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600))
    )
    panels.append(
        pn.Row('Assistant:', pn.pane.HTML(f'<div style="background-color: #F6F6F6;">{response}</div>', width=600))
    )
    
    # Return a Column containing all the conversation panels
    return pn.Column(*panels)

# Event handler for button click
button_conversation.on_click(collect_messages)

# Dashboard layout
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.Column(*panels, height=300),
)

# Display the dashboard
dashboard.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'79398746-1407-4b3f-a470-b3d7aeb8661f': {'version…

# Conclusion
TO BE DRAFTED |